In [10]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch.nn.functional as F

from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

import os

print(os.getcwd()) # dosya yolunu ver
%run ./Model.ipynb
%run ./Dataset.ipynb

plt.rcParams['font.size'] = 14
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

/content


In [11]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [12]:
class Trainer:
    def __init__(self, model, optimizer, device='cpu', max_patience=20):
        self.model = model.to(device)
        self.optimizer = optimizer
        self.device = device

        self.max_patience = max_patience
        self.best_val_loss = float('inf')
        self.patience = 0

        self.val_cost = []
        self.train_cost = []

    def train(self, train_loader, val_loader=None, epochs=10, print_every=1, name=None):
        self.model.train()

        for epoch in range(1, epochs + 1):
            total_loss = 0.0
            for x, x_, y in train_loader:  # label kullanılmıyor
                x = x.to(self.device)
                x_ = x_.to(self.device)

                with torch.no_grad():
                    z_hat_, x_hat_ = self.model(x_)
                z_hat, x_hat = self.model(x)

                loss = self.model.criterion(z_pred=z_hat_, z_true=z_hat, x_pred=x_hat, x_true=x)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()

            # Validation
            if val_loader:
                val_loss = self.validate(val_loader)

                # Early stopping kontrolü
                if val_loss < self.best_val_loss:
                    print('saved!')
                    torch.save(self.model, name + '.model')
                    self.best_val_loss = val_loss
                    self.patience = 0

                else:
                    self.patience = self.patience + 1

                if self.patience > self.max_patience:
                    break

            if epoch % print_every == 0:
                print(f"Epoch {epoch:3d} | Train Loss: {total_loss / len(train_loader):.6f} | Validation Loss: {val_loss:.6f}")

            self.val_cost.append(val_loss)
            self.train_cost.append(total_loss / len(train_loader))

            #torch.cuda.empty_cache()
            #torch.cuda.ipc_collect()

    def validate(self, val_loader):
        self.model.eval()
        total_loss = 0.0

        with torch.no_grad():
            for x, x_, _ in val_loader:
                x = x.to(self.device)
                x_ = x_.to(self.device)

                z_hat_, x_hat_ = self.model(x_)
                z_hat, x_hat = self.model(x)

                loss = self.model.criterion(z_pred=z_hat_, z_true=z_hat, x_pred=x_hat, x_true=x)

                total_loss += loss.item()

        avg_loss = total_loss / len(val_loader)
        #print(f"→ Validation Loss: {avg_loss:.6f}")
        self.model.train()
        return avg_loss

In [13]:
# Custom Transform
class NoiseTransform:
    """Add some noise."""

    def __init__(self, split_ratio=0.001, dim=3072):
        self.split_ratio = split_ratio
    def __call__(self, x):
      return x + split_ratio * torch.randn_like(x)

In [14]:
# Hyper-Parameters & Settings

batch_size = 250
lr = 0.001

epochs = 300
max_patience = 300

split_ratio = 0.0001

In [15]:
# Dataset
train_dataset = CIFAR10Dataset(mode='train', transform=NoiseTransform(split_ratio))
val_dataset = CIFAR10Dataset(mode='val', transform=NoiseTransform(split_ratio))
test_dataset = CIFAR10Dataset(mode='test')

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [16]:
"""
X_list = []
y_list = []

for x, _, y in train_dataset:
    X_list.append(x)
    y_list.append(y)

X = torch.stack(X_list)
y = torch.tensor(y_list)

print('9 dan',X[y==9].shape[0], 'sayıda var')
"""

"\nX_list = []\ny_list = []\n\nfor x, _, y in train_dataset:\n    X_list.append(x)\n    y_list.append(y)\n\nX = torch.stack(X_list)\ny = torch.tensor(y_list)\n\nprint('9 dan',X[y==9].shape[0], 'sayıda var')\n"

In [17]:
# "results" klasörünü oluştur (zaten varsa hata vermez)
os.makedirs("./results", exist_ok=True)

name = 'results/UAE_Cifar_10'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model oluştur
model = To_Uniform(input_channels=3, latent_dim=256, output_channels=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [18]:
# Training
trainer = Trainer(model, optimizer, device=device, max_patience=max_patience)
trainer.train(train_loader, val_loader, epochs=epochs, name=name)

saved!
Epoch   1 | Train Loss: 0.040023 | Validation Loss: 0.025531
saved!
Epoch   2 | Train Loss: 0.023233 | Validation Loss: 0.021703
saved!
Epoch   3 | Train Loss: 0.019144 | Validation Loss: 0.017068
saved!
Epoch   4 | Train Loss: 0.016338 | Validation Loss: 0.015372
saved!
Epoch   5 | Train Loss: 0.014872 | Validation Loss: 0.014227
saved!
Epoch   6 | Train Loss: 0.013582 | Validation Loss: 0.012939
saved!
Epoch   7 | Train Loss: 0.012910 | Validation Loss: 0.012411
saved!
Epoch   8 | Train Loss: 0.011840 | Validation Loss: 0.011110
saved!
Epoch   9 | Train Loss: 0.011114 | Validation Loss: 0.010703
saved!
Epoch  10 | Train Loss: 0.010514 | Validation Loss: 0.010366
Epoch  11 | Train Loss: 0.010114 | Validation Loss: 0.010768
saved!
Epoch  12 | Train Loss: 0.009694 | Validation Loss: 0.009144
Epoch  13 | Train Loss: 0.009347 | Validation Loss: 0.009220
saved!
Epoch  14 | Train Loss: 0.008950 | Validation Loss: 0.008564
saved!
Epoch  15 | Train Loss: 0.008707 | Validation Loss: 0.0

In [19]:
# CSV dosyasına kaydet
train_losses = trainer.train_cost  # liste veya numpy array
val_losses = trainer.val_cost

np.savetxt("results/losses.csv",
           np.column_stack((train_losses, val_losses)),
           delimiter=",",
           header="train_loss,val_loss",
           comments="")